# Multiple Factor Analysis (MFA)

In [1]:
#changement de dossier
import os
os.chdir("d:/Bureau/PythonProject/packages/scientisttools/data/")

#importation des données
import pandas as pd
url = "http://factominer.free.fr/factomethods/datasets/wine.txt"
wine = pd.read_table(url,sep="\t")
wine.head()

,Label,Soil,Odor.Intensity.before.shaking,Aroma.quality.before.shaking,Fruity.before.shaking,Flower.before.shaking,Spice.before.shaking,Visual.intensity,Nuance,Surface.feeling,...,Acidity,Astringency,Alcohol,Balance,Smooth,Bitterness,Intensity,Harmony,Overall.quality,Typical
2EL,Saumur,Env1,3.074,3.000,2.714,2.280,1.960,4.321,4.000,3.269,...,2.107,2.429,2.500,3.250,2.731,1.926,2.857,3.143,3.393,3.250
1CHA,Saumur,Env1,2.964,2.821,2.375,2.280,1.680,3.222,3.000,2.808,...,2.107,2.179,2.654,2.926,2.500,1.926,2.893,2.964,3.214,3.036
1FON,Bourgueuil,Env1,2.857,2.929,2.560,1.960,2.077,3.536,3.393,3.000,...,2.179,2.250,2.643,3.321,2.679,2.000,3.074,3.143,3.536,3.179
1VAU,Chinon,Env2,2.808,2.593,2.417,1.913,2.160,2.893,2.786,2.538,...,3.179,2.185,2.500,2.333,1.680,1.963,2.462,2.038,2.464,2.250
1DAM,Saumur,Reference,3.607,3.429,3.154,2.154,2.040,4.393,4.036,3.385,...,2.571,2.536,2.786,3.464,3.036,2.071,3.643,3.643,3.741,3.444


In [2]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 2EL  to T2  
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Label                          21 non-null     object 
 1   Soil                           21 non-null     object 
 2   Odor.Intensity.before.shaking  21 non-null     float64
 3   Aroma.quality.before.shaking   21 non-null     float64
 4   Fruity.before.shaking          21 non-null     float64
 5   Flower.before.shaking          21 non-null     float64
 6   Spice.before.shaking           21 non-null     float64
 7   Visual.intensity               21 non-null     float64
 8   Nuance                         21 non-null     float64
 9   Surface.feeling                21 non-null     float64
 10  Odor.Intensity                 21 non-null     float64
 11  Quality.of.odour               21 non-null     float64
 12  Fruity                         21 non-null     float

In [3]:
from prince import MFA

In [4]:
wine2 = pd.DataFrame(data=wine.values,
                     columns = pd.MultiIndex.from_tuples(
    [
        ("others","Label"),
        ("others","Soil"),
        ("before shaking","Odor.Intensity"),
        ("before shaking","Aroma.quality"),
        ("before shaking","Fruity"),
        ("before shaking","Flower"),
        ("before shaking","Spice"),
        ("vision","Visual.intensity"),
        ("vision","Nuance"),
        ("vision","Surface.feeling"),
        ("after shaking","Odor.intensity"),
        ("after shaking","Quality.of.odour"),
        ("after shaking","Fruity"),
        ("after shaking","Flower"),
        ("after shaking","Spice"),
        ("after shaking","Plante"),
        ("after shaking","Phenolic"),
        ("after shaking","Aroma.intensity"),
        ("after shaking","Aroma.persistency"),
        ("after shaking","Aroma.quality"),
        ("gustation","Attack.intensity"),
        ("gustation","Acidity"),
        ("gustation","Astringency"),
        ("gustation","Alcohol"),
        ("gustation","Balance"),
        ("gustation","Smooth"),
        ("gustation","Bitterness"),
        ("gustation","Intensity"),
        ("gustation","Harmony"),
        ("overall judgement","Overall.quality"),
        ("overall judgement","Typical")
    ]
))

In [5]:
groups = wine2.columns.levels[0].drop(["others","overall judgement"]).tolist()
groups

['after shaking', 'before shaking', 'gustation', 'vision']

In [6]:
wine2 = wine2[groups].astype("float")

In [7]:
import prince
mfa = prince.MFA(
    n_components=3,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
mfa = mfa.fit(wine2, groups=groups)

In [8]:
mfa.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,3.462,63.59%,63.59%
1,1.367,25.11%,88.70%
2,0.615,11.30%,100.00%


In [11]:
mfa.row_coordinates(wine2).head(6)

,0,1,2
0,-0.238874,-0.796677,0.935737
1,2.044793,-1.383315,1.513530
2,1.220141,-0.459020,0.062333
3,4.381299,0.994551,-0.033460
4,-2.695771,-0.120330,-0.689965
5,-0.868637,-0.326270,0.391083


In [29]:
import numpy as np
np.square(mfa.svd_.s)

array([3.46195044, 1.36676827, 0.61542908])

In [30]:
mfa.svd_.s

array([1.86063173, 1.16908865, 0.78449288])

In [27]:
mfa["after shaking"].eigenvalues_

array([4.70074947, 2.48304844, 1.04639148])

In [13]:
mfa.group_row_coordinates(wine2)

group     after shaking                     before shaking            \
component             0         1         2              0         1   
0             -0.357349 -1.350813  0.731070      -0.407263 -1.279499   
1              2.119630 -1.248869  1.933167       1.757936 -2.756439   
2              1.337619 -0.757873 -0.180293       1.792750  0.246577   
3              4.734360  2.291265  0.785341       3.860684  1.108277   
4             -2.980065 -1.467456 -0.286560      -4.362643  0.893450   
5             -0.716090 -1.917507  0.000330       0.766697 -0.234385   
6             -1.928137 -0.953143 -0.556584      -1.959197 -0.005639   
7             -0.474444  2.332716  2.826376       1.045222  0.848753   
8             -0.398505 -0.812036 -1.268089       1.491293  0.169234   
9              1.037181  0.242903 -0.742514       1.802860 -1.409849   
10            -1.052454  0.100972 -0.195318      -0.839267  0.780554   
11            -0.821375  0.502286 -1.061376      -1.221207  0.556526   
12            -1.898522 -0.769542  0.278028      -1.805001 -2.050432   
13            -1.079296 -0.408037 -0.545191      -0.382491 -1.561365   
14            -1.225418 -1.267450  0.050133      -1.221282 -0.962439   
15            -0.056912 -1.882632 -0.613236      -0.485267 -0.383653   
16            -2.939797 -0.820955 -0.623954      -1.983130 -3.333157   
17             0.159385 -1.235416  0.009459       0.652676 -0.482734   
18             5.230552  0.885286 -0.903714       3.131801 -1.616568   
19             0.414535  4.299930 -1.125991      -1.540887  5.136213   
20             0.895104  4.236369  1.488918      -0.094284  6.336575   

group               gustation                        vision            \
component         2         0         1         2         0         1   
0          2.458382  0.847003 -0.921994  0.260639 -1.037884  0.365596   
1          4.961487  1.466624 -0.648569 -0.194408  2.834983 -0.879381   
2          0.579271  0.318874 -0.873791  0.185176  1.431321 -0.450993   
3          2.445325  4.697356  1.812210 -2.503974  4.232797 -1.233546   
4         -3.308917 -1.960365 -0.356832  0.511548 -1.480013  0.449520   
5          0.570285 -1.611642  0.230690  0.528010 -1.913516  0.616122   
6         -1.611426 -1.650636 -0.386277  0.337252 -0.674551  0.226571   
7          3.829785 -0.873217 -0.213764  0.570149 -0.213913  0.186942   
8         -0.894932 -0.253622 -0.523662  0.224801 -0.575278  0.156286   
9         -1.454126  1.056963 -0.002473 -0.397951  0.911056 -0.328865   
10        -2.381689 -1.087520  0.508834  0.222456  0.026174 -0.083550   
11        -1.215480 -1.014115  0.073573  0.320140 -2.308299  0.865154   
12         0.136252 -1.913285 -1.031722  0.768438 -1.515112  0.394016   
13         0.069480 -2.403623 -0.965676  1.249692 -2.639589  0.908706   
14         0.075537 -1.277413  0.102365  0.402171 -1.097865  0.214265   
15        -1.893250 -0.529680 -0.692965  0.233641  0.567755 -0.210429   
16        -1.137080 -0.601437 -0.047649 -0.285957 -0.097592 -0.120211   
17        -0.407994  0.301276  0.093777 -0.254713  0.746506 -0.332357   
18         0.273806  6.022955 -0.474287 -1.779352  5.158965 -1.546186   
19        -3.337472  0.796988  1.864561 -0.535396 -1.206751  0.379570   
20         2.242755 -0.331484  2.453652  0.137639 -1.149194  0.422771   

group                
component         2  
0          0.292855  
1         -0.646124  
2         -0.334820  
3         -0.860530  
4          0.324071  
5          0.465710  
6          0.176432  
7          0.205041  
8          0.102805  
9         -0.268353  
10        -0.104128  
11         0.720524  
12         0.246895  
13         0.719131  
14         0.086060  
15        -0.175580  
16        -0.173466  
17        -0.301819  
18        -1.104428  
19         0.281313  
20         0.348412

In [10]:
mfa.plot(wine2)

alt.Chart(...)